In [1]:
import os
import sys
import cv2
import shutil
import numpy as np
import pandas as pd

root = './labeled-data'

In [7]:
def xflip(row, dim=(640,480)):
    """
    Flip labels along y-axis (horizontally)
    
    Args:
    ------
    row : ndarray
        row of labels (x,y coordinates)
        
    dim : tuple, default = (640,480)
        resolution of the video
    """
    new_row = np.copy(row)
    new_row[::2] = dim[0] - new_row[::2]
    return new_row

def yflip(row, dim=(640,480)):
    """Flip labels along x-axis (vertically)"""
    new_row = np.copy(row)
    new_row[1::2] = dim[1] - new_row[1::2]
    return new_row

In [15]:
def random_exposure(a=0.4, b=0.4, l_bound=0.5):
    """
    Randomly create a lut table, that adjust exposure of the image
    y = x^gamma, gamma > 0
    gamma is sampled from a beta distribution
    
    Args
    ------
    a : float
        parameter a for the distribution
        
    b : float
        parameter b
        
    l_bound : float
        lower bound to be added to the random number from beta
        
    Returns
    ------
    gamma_table : ndarray
        Lut table in (1 x 256) numpy array
        
    """
    
    gamma = np.random.beta(a, b) + l_bound
    gamma_table = np.power(np.arange(256)/255, gamma) * 255
    gamma_table = np.round(gamma_table).astype(np.uint8)
    return gamma_table

def lut(frame, lut_table): 
    """
    Map LUT table to the image
    
    Args:
    ------
    frame : ndarray
        image to be mapped
        
    lut_table : ndarray (1 x 256)
        lut table to use
    
    """
    return cv2.LUT(frame, lut_table)

In [39]:
def augment(ls, img, row, flipcode, prefix):
    """
    Args
    ------
    ls : list
        list to save elements
    
    img : ndarray
        image
    
    row : ndarray
        labels for the image
        
    flipcode : int
        flipcode, see documentation for 'cv2.flip'
        
    prefix : str
        path to save flipped image
        
    Returns
    ------
    None, 
    - augmented image is wrote into the disk
    - corresponding labels are saved into the [ls]
    
    """
    
    # augment
    new_img = cv2.flip(img, flipcode)  # flip
    lut_table = random_exposure()
    new_img = lut(new_img, lut_table)  # adjust exposure
    
    # metadata
    if flipcode == 1:     # horizontal
        new_row = xflip(row)
        new_name = prefix + "_h.png"
        
    elif flipcode == 0:   # vertical
        new_row = yflip(row)
        new_name = prefix + "_v.png"
        
    elif flipcode == -1:  # both
        new_row = xflip(yflip(row))
        new_name = prefix + "_d.png"
        
    else:
        raise ValueError("Invalid flipcode")

    # save
    cv2.imwrite(new_name, new_img)
    ls.append((new_name, new_row))

In [40]:
from time import sleep

# make directory for augmented data
if os.path.exists("./labeled-data\\aug"):
    print("Removing existing aug folder")
    shutil.rmtree("./labeled-data\\aug")
    try:
        shutil.rmtree("./labeled-data\\aug_labeled")
    except:
        pass
    sleep(20)  # wait for google drive to sync
os.makedirs("labeled-data\\aug")
    
# 
new_csv = []

# iterate thru sub-directories
for path, sub, files in os.walk(root):

    # filter out non-leaf directories
    if not sub:
        
        # find the labels.csv file
        csv = [f for f in files if f.endswith('csv')]
        
        # ignore irrelevant folders
        if csv:
            
            print(f"Processing {path} : ", end='')
            csv = os.path.join(path, csv[0])
            imgs = sorted([i for i in files if 'png' in i])
            lbls = pd.read_csv(csv, header=None, skiprows=[0,1,2,3]).set_index(0)
            
            # augment each image
            for (idx, row), img_name in zip(lbls.iterrows(), imgs):
                
                # check file name
                img_path = idx.split("\\")
                # assert img_path[-1] == img_name, "Unmatched image file"
                prefix = "labeled-data\\aug\\" + ("-".join(img_path[-2:]))[:-4]  # file extension removed
                
                # read image
                img = cv2.imread(os.path.join(path, img_name))
                
                # augment
                augment(new_csv, img, row, 0, prefix)
                augment(new_csv, img, row, 1, prefix)
                augment(new_csv, img, row, -1, prefix)
                
                print("█", end='')
                
            print("\n") 

Removing existing aug folder
Processing ./labeled-data\1 : █████████████████████████████

Processing ./labeled-data\2 : █████████████████████████

Processing ./labeled-data\3 : █████████████████████████

Processing ./labeled-data\4 : ████████████████████

Processing ./labeled-data\5 : ████████████████████

Processing ./labeled-data\6 : ███████████████████████████████████████████████

Processing ./labeled-data\7 : ████████████████████



In [41]:
# copy a template for DeepLabCut to read
headers = pd.read_csv("./labeled-data\\1\\CollectedData_Harris.csv", header=None).iloc[:4].values

# rearrange elements and form the new csv file
indice = np.array(list(zip(*new_csv))[0], ndmin=2).T
values = np.array(list(zip(*new_csv))[1])

new_values = np.hstack((indice, values))
new_values = np.vstack((headers, new_values))

output = pd.DataFrame(new_values)
output  # augmented data path + corresponding labels

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,scorer,Harris,Harris,Harris,Harris,Harris,Harris,Harris,Harris,Harris,...,Harris,Harris,Harris,Harris,Harris,Harris,Harris,Harris,Harris,Harris
1,individuals,mouse1,mouse1,mouse1,mouse1,mouse1,mouse1,mouse1,mouse1,mouse1,...,single,single,single,single,single,single,single,single,single,single
2,bodyparts,snout,snout,left_ear,left_ear,right_ear,right_ear,spine1,spine1,spine2,...,ll_corner,ll_corner,lr_corner,lr_corner,l_screen,l_screen,m_screen,m_screen,r_screen,r_screen
3,coords,x,y,x,y,x,y,x,y,x,...,x,y,x,y,x,y,x,y,x,y
4,labeled-data\aug\1-img02093_v.png,230.1088794926005,277.4435215946843,226.4845967985504,333.01585623678636,191.449864089399,311.27016007248557,185.4093929326488,340.2644216248867,156.41513138024771,...,83.92947749924497,32.20039263062512,591.329054666264,53.94608879492591,176.95273331319845,405.5015101177891,329.17260646330413,415.1662639685895,487.43295077016006,419.99864089398966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557,labeled-data\aug\7-img43399_h.png,259.9689213,417.0804683,243.59505130000002,372.6371068,269.3254185,372.6371068,251.78198630000003,341.0589289,240.08636489999998,...,512.5943447,435.7934626,21.378243699999985,439.3021491,402.655503,67.38138691,252.9515485,70.89007334,98.56934530000001,75.56832192
558,labeled-data\aug\7-img43399_d.png,259.9689213,62.91953169999999,243.59505130000002,107.36289319999997,269.3254185,107.36289319999997,251.78198630000003,138.9410711,240.08636489999998,...,512.5943447,44.2065374,21.378243699999985,40.69785089999999,402.655503,412.61861309,252.9515485,409.10992666,98.56934530000001,404.43167808
559,labeled-data\aug\7-img51373_v.png,383.5397651,86.3107746,416.2875052,115.54982819999998,382.370203,130.75413609999998,410.4396945,149.46713039999997,412.7788187,...,128.5752175,44.2065374,618.6217563,40.69785089999999,233.8358105,411.44905095,387.0484515,407.94036451,539.0915304,402.09255379
560,labeled-data\aug\7-img51373_h.png,256.4602349,393.6892254,223.7124948,364.4501718,257.629797,349.2458639,229.56030550000003,330.5328696,227.2211813,...,511.4247825,435.7934626,21.378243699999985,439.3021491,406.1641895,68.55094905,252.9515485,72.05963549,100.90846959999999,77.90744621


In [42]:
output.to_csv("labeled-data\\aug\\"+"CollectedData_Harris.csv", header=False, index=False)

In [43]:
# calling DeeplabCut to make csv into h5 files
from deeplabcut import convertcsv2h5
convertcsv2h5('config.yaml', userfeedback=False)
print('done')

done
